# Uniswap V3 price analysis

In this notebook we will show how to download price events from Uniswap V3 to your computer as CSV files and use them to analyse price in each pool.

* For more background information, see [this blog post about the topic](https://tradingstrategy.ai/blog/uniswap-data-research-with-jupyter-and-python)

* You need to understand [Jupyter Notebook and Pandas basics](https://jakevdp.github.io/PythonDataScienceHandbook/)

* You need to understand [Ethereum](https://github.com/ethereumbook/ethereumbook) and [Web3.py basics](https://web3py.readthedocs.io/)

* You will need to have [Ethereum API node and its JSON-RPC URL](https://ethereumnodes.com/) in order to pull out the data from Ethereum blockchain. The notebook will interactively ask you for your API key.

* You can click *Launch binder* button on this documentation page to automatically open this notebook to be run in
  [Binder cloud notebook service](https://mybinder.org/). Note that we recommend to run the notebook
  on your local computer, as generated CSV needs are large and take long time to generate.

### Download the raw data from Ethereum blockchain

For simplicity, you can sign up for free access to an Ethereum node for example at [Infura](https://infura.io/), however we recommend you to [run your own Ethereum node](https://tradingstrategy.ai/blog/preparing-a-server-for-hosting-goethereum-full-node).

In [1]:
from web3 import Web3, HTTPProvider

# Get your node JSON-RPC URL
#json_rpc_url = input("Please enter your Ethereum mainnet JSON-RPC URL here")
json_rpc_url = "https://eth-mainnet.g.alchemy.com/v2/h45KpuNet-ON7w9ddDT2UJ7-Kie-tlSt"
json_rpc_url = "https://mainnet.infura.io/v3/eb7d65c7b1334891b7e439353e1aef00"
web3 = Web3(HTTPProvider(json_rpc_url))

As an example, here we download raw events from first few blocks after Uniswap V3 was deployed.
The events will be stored in several CSV files at `/tmp` folder.

Depends on your internet connection and latency to the Ethereum node, the scan might take hours. However it can resume in case there is a crash, as we save the last scanned block in a JSON state file.

In [2]:
from eth_defi.uniswap_v3.constants import UNISWAP_V3_FACTORY_CREATED_AT_BLOCK
from eth_defi.uniswap_v3.events import fetch_events_to_csv
from eth_defi.event_reader.json_state import JSONFileScanState

start_block = UNISWAP_V3_FACTORY_CREATED_AT_BLOCK
end_block = UNISWAP_V3_FACTORY_CREATED_AT_BLOCK + 50000

!rm -r tmp & mkdir tmp
# Stores the last block number of event data we store
state = JSONFileScanState("tmp/uniswap-v3-price-scan.json")

fetch_events_to_csv(json_rpc_url, state, start_block=start_block, end_block=end_block, max_workers=4, output_folder="tmp/")

/home/codespace/.python/current/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


rm: cannot remove 'tmp': No such file or directory
No previous scan done, starting fresh from block 12,369,621, total 50,000 blocks
Scanning block range 12,369,621 - 12,419,621


Block: 12,419,621, events: 249,616, time:2021-05-12: : 50100it [05:15, 158.90it/s]                         

Wrote 0 PoolCreated events
Wrote 230065 Swap events
Wrote 14077 Mint events
Wrote 4619 Burn events


### Analysing Uniswap v3 price formation

In Uniswap V3, you can get the current price of any pool from any given moment using swap events.

In [3]:
import pandas as pd

swap_df = pd.read_csv("tmp/uniswap-v3-swap.csv")

print(f"We have total {len(swap_df):,} swaps in the dataset")

We have total 230,065 swaps in the dataset


Choose a pool to analyse, for example: USDC/ETH 0.3%.
You can find pool addresses [on Uniswap v3 info site](https://info.uniswap.org/#/).

In [5]:
from eth_defi.uniswap_v3.pool import fetch_pool_details

pool_address = "0x8ad599c3A0ff1De082011EFDDc58f1908eb6e6D8"
pool_details = fetch_pool_details(web3, pool_address)

print(pool_details)
print("token0 is", pool_details.token0, "with", pool_details.token0.decimals, "decimals")
print("token1 is", pool_details.token1, "with", pool_details.token1.decimals, "decimals")

Pool 0x8ad599c3A0ff1De082011EFDDc58f1908eb6e6D8 is USDC-WETH, with the fee 0.3000%
token0 is <USD Coin (USDC) at 0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48, 6 decimals, on chain 1> with 6 decimals
token1 is <Wrapped Ether (WETH) at 0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2, 18 decimals, on chain 1> with 18 decimals


Extract the swaps of this pool from the Uniswap v3 full dataset of all pools.

In [6]:
df = swap_df.loc[swap_df.pool_contract_address == pool_address.lower()]
df

,block_number,timestamp,tx_hash,log_index,pool_contract_address,amount0,amount1,sqrt_price_x96,liquidity,tick
19,305599350,2021-05-05T01:56:23,0xce7c3c307d820785caa12938012372fc9366a614a6aa...,26,0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8,-329608,100000000000000,1377932816571815120446551350158799,4303369674465501,195285
21,305606962,2021-05-05T08:23:26,0x9a1c51b0bffbf840948f3b6e3f3e495ba1cd3fa64854...,192,0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8,-164694492,50000000000000000,1378850591292581266780357299649652,4303369674465501,195298
25,305607968,2021-05-05T09:50:51,0xc58715c62a5bf70a6ca09f0e51546d6cad76c8d4fff0...,8,0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8,-329169,100000000000000,1378852426842022799073024911548633,4303369674465501,195298
26,305610871,2021-05-05T11:59:57,0x288c21b8b4fbf449b1d086a06e43b124ac2bc088c3f5...,86,0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8,2,-329169,1378852426842016741051966412054516,4304946248093346,195298
27,305611552,2021-05-05T12:56:56,0x67502d8ba373287f6d301f6baa77c5a5f4c80d0753c3...,257,0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8,1559137299,-467880854065813753,1370241555019945317645788135487819,4304946248093346,195173
...,...,...,...,...,...,...,...,...,...,...
229905,306287987,2021-05-12T12:19:29,0x3d60ccfdf9a73559fd67b3d61c1b8df818fabc3c58d0...,202,0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8,-121343674742,28500000000000000000,1212616016829038047196588117720006,4900548451510204614,192728
229913,306287988,2021-05-12T12:19:42,0xbf1c96166853a2d79bee48e9dd9505271117f97e347d...,299,0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8,-500000000,117479734769371502,1212617910449823497314879623764715,4900548451510204614,192728
229963,306288017,2021-05-12T12:23:08,0x62c06d5e8b33d3d67d07a4ab94703b46a3353f8429c3...,29,0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8,598149816408,-139429800000000000000,1210253165812151065161204850279619,4604186493253165906,192689
230021,306288133,2021-05-12T12:25:28,0xa84bab530b443d6ff2ad53322ec5762671013accd201...,343,0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8,-32588591970,7628000000000000000,1210384033527277456045735847478000,4604186493253165906,192691


In Uniswap V3, we can get pool's price from pool's tick.

In [7]:
def convert_price(row):
    # USDC/WETH pool has reverse token order, so let's flip it WETH/USDC
    tick = row["tick"]
    return pool_details.convert_price_to_human(tick, reverse_token_order=True)

def convert_value(row):
    # USDC is token0 and amount0
    price = float(row["price"])
    return abs(float(row["amount0"])) / (10**pool_details.token0.decimals)

df = df.copy(deep=True)  # https://stackoverflow.com/a/60885847/315168
df["price"] = df.apply(convert_price, axis=1)
df["value"] = df.apply(convert_value, axis=1)
df[["block_number", "timestamp", "tick", "price", "value"]]

,block_number,timestamp,tick,price,value
19,305599350,2021-05-05T01:56:23,195285,3306.010570811264180309745625,0.329608
21,305606962,2021-05-05T08:23:26,195298,3301.715764035195654815931489,164.694492
25,305607968,2021-05-05T09:50:51,195298,3301.715764035195654815931489,0.329169
26,305610871,2021-05-05T11:59:57,195298,3301.715764035195654815931489,0.000002
27,305611552,2021-05-05T12:56:56,195173,3343.244146385107981172857028,1559.137299
...,...,...,...,...,...
229905,306287987,2021-05-12T12:19:29,192728,4269.212622552637615588172193,121343.674742
229913,306287988,2021-05-12T12:19:42,192728,4269.212622552637615588172193,500.000000
229963,306288017,2021-05-12T12:23:08,192689,4285.894225697599469809930042,598149.816408
230021,306288133,2021-05-12T12:25:28,192691,4285.037175412145286631150947,32588.591970


Then we can convert linear price data to [OHLC candles](https://tradingstrategy.ai/docs/glossary.html#term-OHLCV).

In [54]:
agg_dict = {'open': 'first',
          'high': 'max',
          'low': 'min',
          'close': 'last',
          }

freq = "1D"
agg_df = df.set_index("timestamp")["price"].resample(freq).agg(agg_dict)
agg_df["volume"] = df.set_index("timestamp")["value"].resample(freq).sum()
agg_df = agg_df.fillna(method="ffill")

In [55]:
candles = agg_df.copy(deep=True)

### Drawing OHLCV chart

Now we can plot the [OHLC chart using Plotly](https://plotly.com/python/ohlc-charts/).

In [56]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

candlesticks = go.Candlestick(
    x=candles.index,
    open=candles['open'],
    high=candles['high'],
    low=candles['low'],
    close=candles['close'],
    showlegend=False
)

volume_bars = go.Bar(
    x=candles.index,
    y=candles['volume'],
    showlegend=False,
    marker={
        "color": "rgba(128,128,128,0.5)",
    }
)

fig = go.Figure(candlesticks)
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(candlesticks, secondary_y=True)
fig.add_trace(volume_bars, secondary_y=False)
fig.update_layout(title="ETH/USDC pool after Uniswap v3 deployment", height=800)
fig.update_yaxes(title="Price $", secondary_y=True, showgrid=True)
fig.update_yaxes(title="Volume $", secondary_y=False, showgrid=False)
fig.show()